<a href="https://colab.research.google.com/github/anshupandey/Working_with_Large_Language_models/blob/main/WWL_MP_Retail_Product_Description_Generation_final_solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Industry Background

## Company Name: TechRetail Solutions

**TechRetail Solutions** is a leading e-commerce platform that connects vendors with customers worldwide. The company offers a wide range of products across various categories, including electronics, fashion, home appliances, and more. TechRetail Solutions is known for its user-friendly interface, extensive product catalog, and commitment to providing a seamless shopping experience.

# Problem Statement

One of the critical challenges faced by TechRetail Solutions is ensuring that vendors provide high-quality, clear, and accurate product descriptions. Many vendors struggle to write descriptions that meet the platform's standards and format requirements. This inconsistency leads to a suboptimal user experience, as poorly written descriptions can confuse customers, reduce trust, and ultimately impact sales.

The lack of standardized, engaging, and informative product descriptions also hampers the platform's ability to showcase products effectively. To enhance user experience and improve sales, TechRetail Solutions needs a solution that can help vendors generate high-quality product descriptions effortlessly.

# Solution Approach

To address this challenge, TechRetail Solutions aims to leverage Large Language Models (LLMs) to automate the generation of product descriptions. By using advanced LLMs, the company can ensure that product descriptions are not only accurate and informative but also engaging and consistent with the platform's standards.

The solution involves evaluating different LLMs to identify the most suitable model for generating product descriptions. The evaluation will be based on key metrics such as **BLEU**, **ROUGE**, human evaluation, latency, and resource usage. By selecting the best-performing LLM, TechRetail Solutions can provide vendors with a tool that generates high-quality product descriptions, enhancing the overall user experience and boosting sales.

# Dataset Overview

The dataset for evaluating LLMs consists of a diverse set of products across various categories. Each product scenario includes the product name, a brief description, and two high-quality reference descriptions. This dataset will be used to assess the performance of different LLMs based on their ability to generate product descriptions that closely match the reference descriptions.

## Example Dataset:

**Product 1: Wireless Earbuds**

- **Brief Description:** High-fidelity wireless earbuds with noise-canceling technology and long battery life.

- **Reference Description 1:** "Experience the ultimate in wireless freedom with our high-fidelity earbuds. Featuring noise-canceling technology and up to 20 hours of battery life, these earbuds are perfect for music lovers on the go."

- **Reference Description 2:** "Our wireless earbuds offer superior sound quality and comfort. With easy touch controls and a sleek design, enjoy your favorite tunes anytime, anywhere."


Dataset Link: https://github.com/anshupandey/Working_with_Large_Language_models/blob/main/retail_product_description_dataset.json

## Solution: Evaluating LLMs for Retail Product Description Generation

### Objective
Select the best LLM for generating high-quality, engaging, and accurate product descriptions.

### Metrics
- **BLEU**: Aim for a BLEU score above 0.3.
- **ROUGE**: Aim for a ROUGE-L score above 0.5.
- **Perplexity**: Average score less than 20.
- **Latency**: Less than 2 seconds per description.
- **Model Size and Resource Usage**: Fit within available computational resources.

### Steps

### 1. Define Evaluation Criteria
- BLEU score
- ROUGE score
- Perplexity Score
- Latency
- Model Size and Resource Usage

### 2. Benchmarking
- Select candidate LLMs. (Gemini 1.5 Flash, PaLM 2, GPT 35 Turbo)
- Load the benchmark dataset.


### 4. Evaluate LLMs
1. **Generate Descriptions**
   - Use each LLM to generate descriptions.
2. **Calculate Metrics**
   - BLEU Score
   - ROUGE Score
   - Perplexity Score
3. **Analyze Results**
   - Compare metrics.
   - Identify the best-performing model.



## Environment Setup

In [17]:
!pip install together rouge-score --quiet

  Preparing metadata (setup.py) ... done


In [11]:
import os
os.environ["TOGETHER_API_KEY"] = "xxxxxxxxxxxxxxxxxxxx"

In [10]:
import sys
if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

## Load Dta

In [5]:
url = "https://raw.githubusercontent.com/anshupandey/Working_with_Large_Language_models/main/retail_product_description_dataset.json"

In [6]:
import pandas as pd
# load data
df = pd.read_json(url)
df.shape

(10, 3)

In [7]:
df.head()

,name,brief_description,reference_descriptions
0,Wireless Earbuds,High-fidelity wireless earbuds with noise-canc...,Experience the ultimate in wireless freedom wi...
1,Smartwatch,A stylish smartwatch with fitness tracking and...,Stay connected and track your fitness goals wi...
2,Electric Kettle,A 1.7-liter electric kettle with rapid boil te...,Boil water quickly and safely with our 1.7-lit...
3,Gaming Laptop,A high-performance gaming laptop with a powerf...,Unleash your gaming potential with our high-pe...
4,Yoga Mat,"A non-slip, eco-friendly yoga mat with cushion...",Enhance your yoga practice with our eco-friend...


## Setup Prediction Functions

In [8]:
def get_prompt(name,brief_desc):
  prompt = f"""
  for the give product name and brief description, Generate a 2 line product description.
  DO not add any additional information on your own which is not present in the information provided.
  Write the brief description so that its easy to read and interpret,  Add call to action at the end.
  Product Name: {name}
  Brief Description: {brief_desc}
  """
  return prompt

In [12]:
from together import Together
import vertexai

In [13]:
PROJECT_ID = "jrproject-402905"
LOCATION = "us-central1"
MODEL_ID = "gemini-1.5-flash-preview-0514"
vertexai.init(project=PROJECT_ID, location=LOCATION)

In [14]:

from vertexai.generative_models import GenerationConfig, GenerativeModel
gemini_model = GenerativeModel(MODEL_ID)
generation_config = GenerationConfig( temperature=0.9, top_k=32,)

In [22]:
def get_prediction_llama(prompt):
  client = Together()
  response = client.chat.completions.create(
    model="meta-llama/Llama-3-8b-chat-hf",
    messages=[{"role": "user", "content": prompt}],)
  return response.choices[0].message.content


def get_prediction_gemini(prompt):
  contents = [prompt]
  response = gemini_model.generate_content(contents, generation_config=generation_config,)
  return response.text


def get_prediction_gemma(prompt):
  client = Together()
  response = client.chat.completions.create(
    model="google/gemma-7b-it",
    messages=[{"role": "user", "content": prompt}],)
  return response.choices[0].message.content

In [18]:
import math
import nltk
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu

In [19]:
def calculate_perplexity(predicted_sentence, reference_sentence):
    ref_len = len(reference_sentence.split()) # calculating total num of words in reference
    log_prob_sum = 0
    for word in reference_sentence.split():
        if word in predicted_sentence.split():
            log_prob_sum += math.log(1 / (predicted_sentence.split().count(word) / len(predicted_sentence.split())))
        else:
            log_prob_sum += math.log(1 / len(predicted_sentence.split()))
    return math.exp(log_prob_sum / ref_len)


def calculate_bleu(predicted_sentence, reference_sentence):
    return sentence_bleu([reference_sentence.split()], predicted_sentence.split())

# ROUGE Score Calculation
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

def calculate_rouge(predicted_sentence, reference_sentence):
    scores = scorer.score(reference_sentence, predicted_sentence)
    return scores

In [29]:
def get_comparison_report(df):
  print("Evaluating LLMs")
  predictions = {"gemini":[],"llama":[],"gemma":[]}
  reference = []
  import time

  for i in range(len(df)):
    name = df.iloc[i]["name"]
    time.sleep(55)
    brief_desc = df.iloc[i]["brief_description"]
    prompt = get_prompt(name,brief_desc)
    predictions["gemini"].append(get_prediction_gemini(prompt))
    predictions["llama"].append(get_prediction_llama(prompt))
    predictions["gemma"].append(get_prediction_gemma(prompt))
    reference.append(df.iloc[i]["reference_descriptions"].split("|")[0])


  result = {"perplexity":[],"bleu":[],"rouge1":[],"rouge2":[],"rougeL":[]}
  for model in predictions.keys():
    perplexities = [calculate_perplexity(pred, ref) for pred, ref in zip(predictions[model], reference)]
    average_perplexity = sum(perplexities) / len(perplexities)
    result["perplexity"].append(average_perplexity)

    bleus = [calculate_bleu(pred, ref) for pred, ref in zip(predictions[model], reference)]
    average_bleu = sum(bleus) / len(bleus)
    result["bleu"].append(average_bleu)

    rouge_scores = [calculate_rouge(pred, ref) for pred, ref in zip(predictions[model], reference)]

    average_rouge = {
    'rouge1': sum([score['rouge1'].fmeasure for score in rouge_scores]) / len(rouge_scores),
    'rouge2': sum([score['rouge2'].fmeasure for score in rouge_scores]) / len(rouge_scores),
    'rougeL': sum([score['rougeL'].fmeasure for score in rouge_scores]) / len(rouge_scores),
    }
    result["rouge1"].append(average_rouge['rouge1'])
    result["rouge2"].append(average_rouge['rouge2'])
    result["rougeL"].append(average_rouge['rougeL'])
  return pd.DataFrame(result,index=predictions.keys())


In [ ]:
get_comparison_report(df)

Evaluating LLMs
